# markdown.obsidian.links
> Functions for parsing internal links in [Obsidian.md](https://obsidian.md/) style markdown.

Obsidian uses both Markdown style links and Wikilinks as [internal links](https://help.obsidian.md/How+to/Internal+link). Markdown style links are of the form `[text_shown](link)` whereas Wikilinks are of the form `[[link_to_markdown#possible_anchor_to_header|text_shown]]`. They have an exclamation mark `!` if they are embedded.

Within Obsidian, it is often more convenient to use Wikilinks for Vault-internal links for several reasons:

- Obsidian automatically searches for links and aliases matching for auto-completion when constructing a Wikilink.
![Obsidian_link_autocomplete_example.gif](/images/markdown_obsidian_links_Obsidian_link_autocomplete_example.gif)
- Wikilink allow for the empty space character ` ` (whereas Markdown style links require empty space characters ` ` to be replaced with `%20`)

Nevertheless, Markdown style links have the following functions which Wikilinks lack:

- Markdown style links can contain external links (whether the links point to other Obsidian vaults or to a URL)
- Markdown style links can render LaTeX text.


In [ ]:
#| default_exp markdown.obsidian.links

In [ ]:
#| export
from __future__ import annotations
from deprecated import deprecated
from enum import Enum
from pathlib import Path
import re
from trouver.helper import (
    find_regex_in_text, text_from_file
)
from trouver.markdown.obsidian.vault import (
    all_paths_to_notes_in_vault, VaultNote, NoteDoesNotExistError
)
from typing import Union


In [ ]:
#| export
# TODO Make it so that these patterns don't capture latex code
# WIKILINK_PATTERN = r'!?\[\[.*?\]\]' 
WIKILINK_PATTERN = r'!?\[\[[^\]]+\]\]'
EMBEDDED_WIKILINK_PATTERN = r'!\[\[[^\]]+\]\]'
WIKILINK_CAPTURE_PATTERN = r'!?\[\[([^#\|]*?)(#(.*?))?(\|(.*?))?\]\]'

# Note that MARKDOWNLINK_PATTERN captures whitespace characters in its link, even though Obsidian
# does not. This is implmeneted to find if any misformats in the Obsidian Markdown files.
MARKDOWNLINK_PATTERN = r'!?\[[^\]]+\]\([^)]+\)'  
EMBEDDED_MARKDOWNLINK_PATERN = r'!\[[^\]]+\]\([^)]+\)'
MARKDOWNLINK_CAPTURE_PATTERN = r'!?\[([^\]]*)\]\(([^)#]+)(#([^)]+))?\)'


EMBEDDED_PATTERN = f'{EMBEDDED_WIKILINK_PATTERN}|{EMBEDDED_MARKDOWNLINK_PATERN}'
# MARKDOWNLINK_CAPTURE = r'!?\[([^\]]+)\]\(([^)#])+(#[^)]+)?\)'

In [ ]:
from os import PathLike

from fastcore.test import *
from nbdev.showdoc import show_doc

## Finding links in text via indices

In [ ]:
#| export
def find_links_in_markdown_text(
        text: str
        ) -> list[tuple]: # Each tuple is of the form `(a,b)` where `text[a:b]` is an obsidian internal link.
    # TODO: rename this function, say to link_ranges_in_text, 
    # because it is confusing when there is a links_from_text function below.
    """Return ranges in the markdown text string
    where internal links occur.

    **See Also**

    - `links_from_text`
    """
    regex = f'{WIKILINK_PATTERN}|{MARKDOWNLINK_PATTERN}'
    return find_regex_in_text(text, pattern=regex)


`find_links_in_markdown_text` returns a list of indices in a string in which the links are located.

In [ ]:
# TODO: add markdown links to example

In [ ]:
tutorial_text = r'''
This is an Obsidian note. It has some [[this_is_the_note_to_which_the_link_points|links]]!
Links are pretty neat. They can [[this_text_is_not_actually_shown|connect notes]] for you.
The following will create a link to the note `some_note`; the displayed text is `some_note`: [[some_note]]
You can also embed the contents of one note into another note. ![[note_being_embedded]].
The contents of `note_being_embedded` will be displayed when you view the note in Obsidian's view mode.
You can make anchors in links. For example [[note#This is a header title]] is a link to the note named
`note` and more specifically to the theader with title `This is a header title`.

The above links are all Wikilinks. Obsidian also supports Markdownlinks, e.g. [This is the text shown](This is the link.)

If the note of a link does not exist in an Obsidian vault, then Obsidian will create the note.
Even if the note does not have a header with title specified by the anchor of a link, Obsidian
will still open the note; it will not go to any particular header, however.
'''

ranges = find_links_in_markdown_text(tutorial_text)
match_strs = [tutorial_text[start:end] for start, end in ranges]
test_eq(match_strs, [
    '[[this_is_the_note_to_which_the_link_points|links]]', 
    '[[this_text_is_not_actually_shown|connect notes]]',
    '[[some_note]]',
    '![[note_being_embedded]]',
    '[[note#This is a header title]]',
    '[This is the text shown](This is the link.)'])

### Longer Example

The following example is of a note whose contents are based on an excerpt from Vakil's *The Rising Sea - Foundations of Algebraic Geometry*.

In [ ]:
text = r'''
---
cssclass: clean-embeds
aliases: [foag_relative_cotangent_sheaf, foag_relative_tangent_sheaf]
tags: [_meta/definition, _meta/notation, _meta/literature_note, _reference/vakil_rising_sea, relative_cotangent_sheaf, relative_tangent_sheaf, diagonal_map/scheme, conormal_sheaf, dual/coherent_sheaf, morphism/schemes]
---
# Global definition[^1]
Let $\pi: X \rightarrow Y$ be a morphism of schemes. Define the **relative cotangent sheaf** $\Omega_{X/Y}$ or $\Omega_{\pi}$ as the [[foag_conormal_sheaf_of_a_locally_closed_embedding#For a locally closed embedding 2 4|conormal sheaf]] $\mathscr{N}^\vee_{X/X \times_Y X}$ of the diagonal[^2].

# Relative tangent sheaf
Define the **relative tangent sheaf** $\mathscr{T}_{X/Y}$ as the [[foag_dual_sheaf|dual]] $\mathcal{Hom}(\Omega_{X/Y}, \mathscr{O}_X)$[^3]

[^3]: ![[foag_notation_Hom_sheaf_hom]]

# Other
We now define $\mathrm{d}: \hat{\partial}_{\mathrm{X}} \rightarrow \Omega_{\mathrm{X} / \mathrm{Y}}$. Let $\operatorname{pr}_{1}: X \times_{\mathrm{Y}} X \rightarrow X$ and $\operatorname{pr}_{2}: X \times_{\mathrm{Y}} X \rightarrow X$ be the two projections. Then define $\mathrm{d}: \mathscr{O}_{\mathrm{X}} \rightarrow \Omega_{\mathrm{X} / \mathrm{Y}}$ on the open set $\mathrm{U}$ as follows:
$$
d f=p r_{2}^{*} f-p r_{1}^{*} f
$$
(Warning: this is not a morphism of quasicoherent sheaves on $X$, although it $i$ s $\mathscr{O}_{\mathrm{Y}}$-linear in the only possible meaning of that phrase.) We will soon see that $\mathrm{d}$ is indeed a derivation of the sheaf $\mathscr{O}_{\mathrm{X}}$ (in the only possible meaning of the phrase), and at the same time see that our new notion of differentials agrees with our old definition on affine open sets, and hence globalizes the definition. Note that for any open subset $U \subset X, d$ induces a map
$$
\Gamma\left(\mathrm{U}, \mathscr{O}_{\mathrm{X}}\right) \rightarrow \Gamma\left(\mathrm{U}, \Omega_{\mathrm{X} / \mathrm{Y}}\right)
$$
which we also call d, and interpret as "taking the derivative".

# See Also
- [[foag_notation_T_X_Y_relative_tangent_sheaf]]
- [[foag_notation_Omega_X_Y_relative_cotangent_sheaf]]
- [[foag_21.2.Q]]
- [[foag_sheaf_of_relative_i_forms]]

# Meta
## References
![[_reference_foag]]

## Citations and Footnotes
[^1]: Vakil, 21.2.20, Page 572
[^2]: Note that the diagonal morphism is a locally closed embedding.
'''

ranges = find_links_in_markdown_text(text)
for match_range in ranges:
    print(text[match_range[0]:match_range[1]])

[[foag_conormal_sheaf_of_a_locally_closed_embedding#For a locally closed embedding 2 4|conormal sheaf]]
[[foag_dual_sheaf|dual]]
![[foag_notation_Hom_sheaf_hom]]
[[foag_notation_T_X_Y_relative_tangent_sheaf]]
[[foag_notation_Omega_X_Y_relative_cotangent_sheaf]]
[[foag_21.2.Q]]
[[foag_sheaf_of_relative_i_forms]]
![[_reference_foag]]


## `ObsidianLink` class

In [ ]:
#| export
class LinkFormatError(Exception):
    """Error that is raised when a string cannot be parsed as an
    `ObsidianLink` object.
    
    **Attribute**

    - `text` - `str`
    """
    def __init__(self, text):
        self.text = text
        super().__init__(f'Obsidian Markdown link is not formatted properly: {text}')

In [ ]:
#| export
class LinkType(Enum):
    """An Enumeration indicating whether an `ObsidianLink` object is a
    Wikilink or a Markdown-style link.

    Enumerates `LinkType.WIKILINK` and `LinkType.MARKDOWN`.
    """
    # See https://www.markdownguide.org/basic-syntax/
    WIKILINK = 0
    MARKDOWN = 1  
    # For Markdown links, use %20 to encode spaces in the link, e.g.
    # [asdf](localization_of_a_module#Localization%20of%20a%20module%201)
    # Links to the header `"Localization of a module 1"` in the file
    # localization_of_a_module



In [ ]:
#| export
# TODO: implment equality, copy
class ObsidianLink:
    """Object representing an obsidian link
    
    **Attributes**

    - `is_embedded` - `bool`
        - Whether or not the link is embedded.
    - `file_name` - `str`, or `-1`
        - The destination of the link. It is either 
        
          1. The Obsidian-vault-recognized name of the file that the link
          points to. It can be a path relative to the Obsidian vault path 
          without the file extension (.md), 
          2. an external link, such as a URL, or
          3. -1, in which case the object represents a generic link pointing
          to any file (this is for generating regex).
          
          Note that if `file_name` is the empty string, then the link is a
          link to the same file

    - `anchor` - `str`, `0`, or `-1`
        - The title of the header of the anchor in the destination that the
        link points to or the ID to the markdown block link (preceded by a
        carat `^`). If 0, then the `ObsidianLink` object represents a link
        without an anchor. If -1, then the object represents a generic link
        with or without an anchor (this is for generating regex).
    - `custom_text` - `str`, `0`, or `-1`
        - The custom text of the link. Is `None` if no such text is specified.
        If 0, then the `ObsidianLink` object represents an internal link
        without custom text. If -1, then the object represents a generic
        internal link of any custom text (this is for generating regex).
    - `link_type` - `LinkType`
        - If `LinkType.WIKILINK`, then the str should be of the format
        `'[[<Obsidian-vault-recognized-name>(#anchor)?(|custom_text)]]'` 
        (The question marks here indicate optional components). Otherwise,
        the str should be a more standard Markdown link. Defaults to
        `LinkType.WIKILINK`.
    
    **Parameters**

    - is_embedded - bool
    - file_name - str or `None`
        - If `None`, set `self.file_name` to `-1`.
    - anchor - str or `None`
    - custom_text - str or `None`
    - link_type - `LinkType`
    """
    
    def __init__(
            self, is_embedded: bool, file_name: Union[str, int],
            anchor: Union[str, int], custom_text: Union[str, int],
            link_type: LinkType = LinkType.WIKILINK):
        self.is_embedded = is_embedded
        self.file_name = file_name
        self.anchor = anchor
        self.custom_text = custom_text
        self.link_type = link_type


    @staticmethod
    def from_text(text: str) -> ObsidianLink:
        """Return an ObsidianLink object from text.
                
        **Raises**

        - InteralLinkFormatError
            - If `text` is not properly formatted as an Obsidian internal link.
        """
        is_embedded = text.startswith("!")
        regex_object = re.compile(WIKILINK_CAPTURE_PATTERN)
        matches = regex_object.match(text)
        if matches:
            file_name = matches.group(1)
            anchor = matches.group(3)
            custom_text = matches.group(5)
            link_type = LinkType.WIKILINK
        else:
            regex_object = re.compile(MARKDOWNLINK_CAPTURE_PATTERN)
            matches = regex_object.match(text)
            if not matches:
                raise LinkFormatError(text)
            file_name = matches.group(2).replace('%20', ' ')
            anchor = matches.group(4)
            if anchor:
                anchor = anchor.replace('%20', ' ')
            custom_text = matches.group(1)
            link_type = LinkType.MARKDOWN
        if anchor is None:
            anchor = 0
        if custom_text is None:
            custom_text = 0
        return ObsidianLink(is_embedded, file_name, anchor, custom_text, link_type)

    def _parse_text_as_wikilink(text: str):
        """
        Return details about the link `text` if `text` is a Wikilink or `None` if
        `text is not a Wikilink.

        This is a helper method for `from_text`.
        """
        # TODO
        return

    def to_regex(self
            )-> str: # Represents a regex.
        """Return the regex for that this `ObsidianLink` object represents.

        Assumes that `self.file_name`, `self.anchor`, and `self.custom_text` are
        regex-formatted strings, e.g. if `self.custom_text` is `denotes?`, then the
        outputted regex-pattern matches links whose custom text is either `denote`
        or `denotes`.

        If neither `self.file_name`, `self.anchor` nor `self.custom_text` is `-1`,
        then the regex will in fact be a concrete string.
        """
        embedding = '!' if self.is_embedded else ''

        if type(self.file_name) == str:
            filing = self.file_name
        else:  # self.file_name == -1
            filing = r'([^#\|]*)?'
        
        if type(self.anchor) == str:
            anchoring = f'#{self.anchor}'
        elif self.anchor == 0:
            anchoring = ''
        else:  # self.anchor == -1
            anchoring = '(#(.*?))?'
          
        if type(self.custom_text) == str and self.link_type == LinkType.WIKILINK:
            customing = fr'\|{self.custom_text}'
        elif type(self.custom_text) == str and self.link_type == LinkType.MARKDOWN:
            customing = self.custom_text
        elif self.custom_text == 0:
            customing = ''
        else:  # self.custom == -1
            if self.link_type == LinkType.MARKDOWN:
                customing = fr'(.*?)?'
            else:
                customing = fr'(\|(.*?))?'

        if self.link_type == LinkType.WIKILINK:
            return fr'{embedding}\[\[{filing}{anchoring}{customing}\]\]'
        else:
            # Markdown links format whitespace with '%20'
            filing = filing.replace(' ' , '%20')  
            anchoring = anchoring.replace(' ', '%20')
            return fr'{embedding}\[{customing}\]\({filing}{anchoring}\)'
    
    def __str__(self) -> str:
        # TODO: Choose what to do about | vs. \|.
        return self.to_string()

    def to_string(self
            ) -> str: # The string for the link
        """
        Return the string for the link if it is concrete.
 
        **Raises**

        - ValueError
            - If `self.file_name`, `self.anchor` or `self.custom_text`
            is -1, i.e. ambiguously represents an anchor or custom text.
        """
        if self.is_abstract():
            raise ValueError(
                f'The ObsidianLink object is abstract.'
            )
        assert (self.anchor != -1 and self.custom_text != -1
                and self.file_name != -1)
        embedding = '!' if self.is_embedded else ''

        if type(self.anchor) == str:
            anchoring = f'#{self.anchor}'
        else:  # self.anchor == 0
            anchoring = ''
          
        if type(self.custom_text) == str:
            if self.link_type == LinkType.WIKILINK:
                customing = fr'|{self.custom_text}'
            else:
                customing = self.custom_text
        else:  # self.custom_text == 0:
            customing = ''
        
        if self.link_type == LinkType.WIKILINK:
            return f'{embedding}[[{self.file_name}{anchoring}{customing}]]'
        else:
            # Markdown links format whitespace with '%20'
            file_name = self.file_name.replace(' ' , '%20')  
            anchoring = anchoring.replace(' ', '%20')
            return fr'{embedding}[{customing}]({file_name}{anchoring})'
    
    def convert_link_type(
            self,
            link_type: LinkType
            ) -> ObsidianLink:
        """
        Return an equivalent Link object which has the specified
        `LinkType`.
        """
        # TODO
        return
    
    def displayed_text(self
            ) -> str: # The displayed text
        # TODO: implement error if any of the attributes is -1
        """Returns the displayed text of this link.
        
        `self.file_name`, `self.custom_text` and `self.anchor` are
        assumed to be not `-1`.
        """
        if self.custom_text:
            return self.custom_text
        else:
            if not self.anchor:
                return self.file_name
            else:
                return f'{self.file_name} > {self.anchor}'

    def is_abstract(self) -> bool:
        """
        Return `True` if self is abstract, i.e. file_name, anchor,
        or custom_text is `-1`.
        """
        return self.anchor == -1 or self.file_name == -1 or self.anchor == -1
    

In [ ]:
show_doc(ObsidianLink.from_text)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L146){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.from_text

>      ObsidianLink.from_text (text:str)

Return an ObsidianLink object from text.

**Raises**

- InteralLinkFormatError
    - If `text` is not properly formatted as an Obsidian internal link.

In [ ]:
show_doc(ObsidianLink.to_regex)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L180){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.to_regex

>      ObsidianLink.to_regex ()

Return the regex for that this `ObsidianLink` object represents.

Assumes that `self.file_name`, `self.anchor`, and `self.custom_text` are
regex-formatted strings, e.g. if `self.custom_text` is `denotes?`, then the
outputted regex-pattern matches links whose custom text is either `denote`
or `denotes`.

If neither `self.file_name`, `self.anchor` nor `self.custom_text` is `-1`,
then the regex will in fact be a concrete string.

In [ ]:
show_doc(ObsidianLink.__str__)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L229){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.__str__

>      ObsidianLink.__str__ ()

Return str(self).

In [ ]:
show_doc(ObsidianLink.to_string)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L233){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.to_string

>      ObsidianLink.to_string ()

Return the string for the link if it is concrete.

**Raises**

- ValueError
    - If `self.file_name`, `self.anchor` or `self.custom_text`
    is -1, i.e. ambiguously represents an anchor or custom text.

In [ ]:
show_doc(ObsidianLink.convert_link_type)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L273){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.convert_link_type

>      ObsidianLink.convert_link_type (link_type:__main__.LinkType)

Return an equivalent Link object which has the specified
`LinkType`.

In [ ]:
show_doc(ObsidianLink.displayed_text)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L283){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.displayed_text

>      ObsidianLink.displayed_text ()

Returns the displayed text of this link.

`self.file_name`, `self.custom_text` and `self.anchor` are
assumed to be not `-1`.

In [ ]:
show_doc(ObsidianLink.is_abstract)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/links.py#L297){target="_blank" style="float:right; font-size:smaller"}

### ObsidianLink.is_abstract

>      ObsidianLink.is_abstract ()

Return `True` if self is abstract, i.e. file_name, anchor,
or custom_text is `-1`.

#### Parsing an ObsidianLink class from a string

The following example parses a Wikilink with an anchor and a custom display text:

In [ ]:
internal_link = ObsidianLink.from_text("[[smooth_covering_map#Smooth covering map 1|smooth covering map]]")
assert internal_link.file_name == "smooth_covering_map"
assert internal_link.anchor == "Smooth covering map 1"
assert internal_link.custom_text == "smooth covering map"

The following example parses a Wikilink with an anchor, but not with a custom display text:

In [ ]:
internal_link = ObsidianLink.from_text("[[sample_thing#anchor]]")
assert internal_link.file_name == "sample_thing"
assert internal_link.anchor == "anchor"
assert internal_link.custom_text == 0

The following example parses a Wikilink without an anchor, but with custom display text:

In [ ]:
internal_link = ObsidianLink.from_text("[[another_file|this_is_the_text_shown]]")
assert internal_link.file_name == "another_file"
assert internal_link.anchor == 0
assert internal_link.custom_text == "this_is_the_text_shown"

The following example parses a Wikilink without neither an anchor nor custom display text:

In [ ]:
internal_link = ObsidianLink.from_text("[[foag_notation_I_S_ideal_of_vanishing_on_a_subset_of_the_spectrum_of_a_ring]]")
assert internal_link.file_name == "foag_notation_I_S_ideal_of_vanishing_on_a_subset_of_the_spectrum_of_a_ring"
assert internal_link.anchor == 0
assert internal_link.custom_text == 0

The following example parses an embedded (Wiki)link

In [ ]:
internal_link = ObsidianLink.from_text("![[_reference_foag]]")
assert internal_link.file_name == "_reference_foag"
assert internal_link.anchor == 0
assert internal_link.custom_text == 0

The following example parses a Markdownlink with an anchor:

In [ ]:
internal_link = ObsidianLink.from_text("[asdf](localization_of_a_module#Localization of a module 1)")
assert internal_link.file_name == "localization_of_a_module"
assert internal_link.anchor == "Localization of a module 1"
assert internal_link.custom_text == "asdf"

The following example parses a Markdownlink with some spaces formatted as `%20`:

In [ ]:
internal_link = ObsidianLink.from_text(r"[do do](foag_12.2%20Exercise#Topic%201)")
assert internal_link.file_name == "foag_12.2 Exercise"
assert internal_link.anchor == "Topic 1"
assert internal_link.custom_text == "do do"

The following example parses a Markdownlink in which the displaytext has LaTeX in it:

In [ ]:
internal_link = ObsidianLink.from_text("[$\\mathscr{O}(n)$](foag_notation_O_n_on_projective_scheme)")
assert internal_link.file_name == "foag_notation_O_n_on_projective_scheme"
assert internal_link.anchor == 0
assert internal_link.custom_text == "$\\mathscr{O}(n)$"
# TODO test links with file_name = -1

#### Constructing abstract ObsidianLink objets

We might want to detect links in Obsidian Markdown notes of certain formats. Constructing `ObsidianLink` objects via the constructor can help with this.

Setting `file_name`, `anchor`, or `custom_text` to `-1` can yields an `ObisidianLink` object representing an abstract link. Use the `to_regex` function of the `ObsidianLink` object to get a regex str which detects links of the specified format.

In the following example, the anchor and custom text of the `ObsidianLink` object are both abstract - the regex pattern returned by `._to_regex` will detect any link of the specified type (by default, `LinkType.WIKILINK`) with the specified file name:

In [ ]:
internal_link_object = ObsidianLink(is_embedded=False, file_name = 'hi', anchor=-1, custom_text=-1)
regex_pattern = internal_link_object.to_regex()
print(f'regex_pattern: {regex_pattern}')
assert re.match(regex_pattern, '[[hi#this is some anchor|this is some display text]]')
assert re.match(regex_pattern, '[[hi|some display text, but no anchor!]]')
assert re.match(regex_pattern, '[[hi#some anchor, but no custom text]]')
assert re.match(regex_pattern, '[[hi]]')  #No anchor and no custom text
assert not re.match(regex_pattern, '[[note_with_wrong_name]]')

regex_pattern: \[\[hi(#(.*?))?(\|(.*?))?\]\]


The following is an example where the custom_text is specified, but the file_name and anchor are both abstract:

In [ ]:
internal_link_object = ObsidianLink(is_embedded=False, file_name = -1, anchor=-1, custom_text='must match this text!')
regex_pattern = internal_link_object.to_regex()
print(f'regex_pattern: {regex_pattern}')
assert re.match(regex_pattern, '[[some note#some anchor|must match this text!]]')
# TODO: matching the below is not currently implemented.
# assert re.match(regex_pattern, '[[must match this text!]]')
assert re.match(regex_pattern, '[[some note|must match this text!]]')
assert re.match(regex_pattern, '[[#anchor in note|must match this text!]]')  #No anchor and no custom text
assert not re.match(regex_pattern, '[[some note#some anchor|Wrong text]]')

regex_pattern: \[\[([^#\|]*)?(#(.*?))?\|must match this text!\]\]


In the following example, the linktype is specified to be `LinkeType.MARKDOWN`:

In [ ]:
internal_link_object = ObsidianLink(is_embedded=False, file_name = 'hi', anchor=-1, custom_text=-1, link_type=LinkType.MARKDOWN)
regex_pattern = internal_link_object.to_regex()
print(f'regex_pattern: {regex_pattern}')
assert re.match(regex_pattern, '[Some custom text](hi#Some anchor)')
assert re.match(regex_pattern, '[Some custom text, no anchor](hi)')
assert not re.match(regex_pattern, '[Some custom text, link an anchor in the same note](#anchor)')

regex_pattern: \[(.*?)?\]\(hi(#(.*?))?\)


### String of an ObsidianLink object

A concrete `ObsidianLink` object has a `to_string` function:

In [ ]:
internal_link_object = ObsidianLink(is_embedded=True, file_name = 'compactly_generated_product_hom_adjunction', anchor='Title', custom_text='compactly generated', link_type=LinkType.WIKILINK)
print(internal_link_object.to_string())
internal_link_object = ObsidianLink(is_embedded=True, file_name = 'compactly_generated_product_hom_adjunction', anchor='Title 1', custom_text='compactly generated', link_type=LinkType.MARKDOWN)
print(internal_link_object.to_string())

![[compactly_generated_product_hom_adjunction#Title|compactly generated]]
![compactly generated](compactly_generated_product_hom_adjunction#Title%201)


## Find the links in text as ObsidianLink objects

In [ ]:
#| export
def links_from_text(
        text: str
        ) -> list[ObsidianLink]: # The `ObsidianLink` objects are ordered by appearance.
    """
    Return a list of `ObsidianLink` objects corresponding to links
    found in the text.
    """
    ranges = find_links_in_markdown_text(text)
    link_strs = [text[start:end] for start, end in ranges]
    return [ObsidianLink.from_text(link_str) for link_str in link_strs]

Unlike `find_links_in_markdown_text`, the `links_from_text` function returns a list of `ObsidianLink` objects instead.

In [ ]:
links = links_from_text(tutorial_text)
assert links[0].displayed_text() == 'links'  # The displayed text of [[this_is_the_note_to_which_the_link_points|links]] is `links`.
assert links[3].is_embedded  # ![[note_being_embedded]] is an embedded link
assert links[4].anchor == 'This is a header title'  # [[note#This is a header title]] has anchor `This is a header title`.

## Removing links from text

In [ ]:
#| export
def remove_links_from_text(
        text: str,
        exclude: list[ObsidianLink] = None, # A list of `ObsidianLink` objects of links to not be removed.
        remove_embedded_note_links: bool = False # If `True`, remove links to embedded notes as well. Note that embedded links are replaced by their "display" text in the same manner as non-embedded links and are not replaced the content of the embedding. If `False`, does not modify embedded note links.
        ) -> str:
    """
    Return a text with all Obsidian links removed and replaced with
    the display texts of the links.
    """
    if not exclude:
        exclude = []
    exclude_patterns = [re.compile(exclude_link.to_regex())
                        for exclude_link in exclude]
    link_indices = find_links_in_markdown_text(text)
    new_text = text
    for start, end in reversed(link_indices):
        if _do_not_remove_link(text[start:end], exclude_patterns):
            continue
        link_object = ObsidianLink.from_text(text[start:end])
        if link_object.is_embedded and not remove_embedded_note_links:
            continue
        replace_with = link_object.displayed_text()
        # link_object.custom_text if link_object.custom_text else link_object.file_name
        new_text = new_text[0:start] + replace_with + new_text[end:]
    return new_text


def _do_not_remove_link(text: str, exclude_patterns: list[re.Pattern]) -> bool:
    """
    Return `True` if text fully matches any of `re.Pattern`'s in `exclude_patterns`.

    This is a helper method for `remove_links_from_text`. 
    """
    for exclude_pattern in exclude_patterns:
        if exclude_pattern.fullmatch(text):
            return True
    return False

We can remove all links of from text and preserve the underlying display text of each link:

In [ ]:
text = 'I have a [[this is a note#this is an anchor in the note|link]]'
links_removed = remove_links_from_text(text)
print(links_removed)
assert links_removed == 'I have a link'

text = 'Something about a [[foag_conormal_sheaf_of_a_locally_closed_embedding#For a locally closed embedding 2 4|conormal sheaf]]'
links_removed = remove_links_from_text(text)
print(links_removed)
assert links_removed == 'Something about a conormal sheaf'

text = 'This is a link without a specified display text: [[some_kind_of_note]].'
links_removed = remove_links_from_text(text)
print(links_removed)
assert links_removed == 'This is a link without a specified display text: some_kind_of_note.'

text = 'This is a link to an anchor without a specified display text: [[another_note#another anchor]].'
links_removed = remove_links_from_text(text)
print(links_removed)
assert links_removed == 'This is a link to an anchor without a specified display text: another_note > another anchor.'


I have a link
Something about a conormal sheaf
This is a link without a specified display text: some_kind_of_note.
This is a link to an anchor without a specified display text: another_note > another anchor.


We can specify links to not remove in this process:

In [ ]:
text = 'This thing [[fulton_it_ B.4.1|denotes]] something'
do_not_remove_link = ObsidianLink(False, None, -1, 'denote(s)?')
links_removed = remove_links_from_text(text, exclude=[do_not_remove_link])
assert text == links_removed

This function does not remove embedded links by default.

In [ ]:
text = '![[embedded note]]'
links_removed = remove_links_from_text(text)
print(links_removed)
assert links_removed == text

![[embedded note]]


In [ ]:
text = '![[embedded note]]'
links_removed = remove_links_from_text(text, remove_embedded_note_links=True)
print(links_removed)
assert links_removed == 'embedded note'

embedded note


## Queries about links in a vault

In [ ]:
#| export
def all_custom_text_for_links_in_vault(
        note: VaultNote, # The note to find the custom text for.
        vault: PathLike, # The path to the Obsidian vault directory
        anchor: Union[str, int] = -1 # The anchor in the note to find the custom text for. If 0, then returns only the custom texts for internal links without anchors. If -1, then returns the custom texts for all of the internal links of the specified note.
        ) -> dict[Union[str, int], str]: # Each key is the custom text used and each value is a set of paths relative to `vault` to the notes where the custom text is used.  The key `0` means that no custom text is used.
    """
    Return all custom text used in the Obsidian vault for the specified
    note and anchor.
    """
    vault = Path(vault)
    name = note.name
    wiki_gen = ObsidianLink(
        is_embedded=False, file_name=name, anchor=anchor, custom_text=-1,
        link_type=LinkType.WIKILINK)
    wiki_regex = wiki_gen.to_regex()
    mark_gen = ObsidianLink(
        is_embedded=False, file_name=name, anchor=anchor, custom_text=-1,
        link_type=LinkType.MARKDOWN)
    mark_regex = mark_gen.to_regex()
    regex = f'{wiki_regex}|{mark_regex}'
    all_note_paths = all_paths_to_notes_in_vault(vault)
    # I am creating a generator as opposed to a list so that the program
    # does not store the contents of all the text files.
    texts = ((path, text_from_file(vault / path, encoding='utf8'))
             for path in all_note_paths)
    regex_object = re.compile(regex)
    custom_text_usage = {}
    for path, text in texts:
        custom_texts = _custom_text_for_links_in_text(text, regex_object)
        for custom_text in custom_texts:
            if custom_text not in custom_text_usage:
                custom_text_usage[custom_text] = set()
            custom_text_usage[custom_text].add(path)
    return custom_text_usage


def _custom_text_for_links_in_text(
        text: str, regex_object: re.Pattern) -> list[str]:
    """Helper function for finding custom text for links in a single file.
    
    **Parameters**
    - text - str
    - regex_object - re.Pattern
        - An object returned with the `re.compile` function. Matches
        wikilinks and markdown links.
        
    **Returns**
    - list[str]
    """
    link_matches = regex_object.finditer(text)
    match_ranges = [match.span() for match in link_matches]
    links = [ObsidianLink.from_text(text[start:end])
             for start, end in match_ranges]
    return [link.custom_text for link in links]

In [ ]:
# TODO: examples

In [ ]:
#| export
def all_links_in_vault(
        vault: PathLike, backlinks: bool = False, 
        multiplicities: bool = False) -> dict[str, list[str]]:
    """Returns a dict keeping track of which notes contain 
    links to which notes.
    
    TODO: currently, the regex capture regex code. Change them
    so that this does not happen.
    
    **Parameters**
    - vault - PathLike
    - backlinks - bool
        - If `True`, then keeps track of the links in each note.
        If `False`, then keeps track of the backlinks in each note,
        i.e. which other notes link to each note. Defaults to `False`.
    - multiplicties - bool
        - If `True`, then keeps track of multiple links to the same note
        for each note. Otherwise, only keeps track of whether or not
        a note links to a(nother) note. Defaults to `False`.
  
    **Returns**
    - dict[str, list[str]]
        - Each key is a str, referring to the name of a note. 
        Each corresponding value is a list. The list contains 
        the names of all the notes which the key note references to if
        `backlinks` is `True`, and contains the names of all the notes which
        reference the key note if `backlinks` is `False`. The list can contain
        multiple occurrences of the same note if `multiplicities` is `True`.
    """
    vault = Path(vault)
    all_note_paths = all_paths_to_notes_in_vault(vault, as_dict=True)
    links = {}
    for name, path in all_note_paths.items():
        text = text_from_file(vault / path, encoding='utf8')
        try:
            links_in_note = links_from_text(text)
        except LinkFormatError:
            print(name)
        if not multiplicities:
            links_in_note = set(links_in_note)
        if backlinks:
            if name not in links:
                links[name] = []
            for link in links_in_note:
                if link.file_name not in links:
                    links[link.file_name] = []
                links[link.file_name].append(name)
        else:
            if name not in links:
                links[name] = []
            links[name] += [link.file_name for link in links_in_note]
    return links

In [ ]:
# TODO: examples